In [1]:
import cv2
import numpy as np
import time
import os
import pandas as pd

In [2]:
#loading the labels
labelspath = './assets/coco.names'
labels = open(labelspath).read().strip().split('\n')

#loading the weights and the configuration file
weights = './assets/yolov3.weights'
config = './assets/yolov3.cfg'

In [3]:
threshold = 0.5
confidence_threshold = 0.6

net = cv2.dnn.readNetFromDarknet(config, weights)
layer_names = net.getLayerNames()
layer_names = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

#loading the image
imagepath = './assets/shutterisland.png'
image = cv2.imread(imagepath)
h, w = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(layer_names)

boxes = []
confidences = []
classIDs = []

for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]

        if confidence > confidence_threshold:
            box = detection[0:4] * np.array([w, h, w, h])
            (centerX, centerY, width, height) = box.astype('int')

            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

##Non max-supression to remove the overlapping bounding boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, threshold)

if len(idxs)>0:
    for i in idxs.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        # np.random.seed(42)
        color = [int(c) for c in np.random.randint(0, 255, size=(3,))]
        cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
        text = '{}: {:.4f}'.format(labels[classIDs[i]], confidences[i])
        cv2.putText(image, text, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

cv2.imwrite('output.png', image)
cv2.imshow('Image', image)
cv2.waitKey(0)

-1